# Display Stations

Before we start, we import some preliminary libraries.

In [1]:
%pip install folium
%pip install openpyxl

In [1]:
import folium
from folium.plugins import TimestampedGeoJson
import matplotlib.pyplot as plt

from datetime import datetime
from collections import defaultdict
import numpy as np
import pandas as pd

from glob import glob
import os
notebook_path = os.path.abspath("DisplayStations.ipynb")

## Functions to load dataframes

In [2]:
RENAME_DICT = {
                "Latitude station prise": "latitude",
                "Longitude station prise": "longitude",
                "Nom station prise": "station_name",
                "Latitude station dépose": "latitude",
                "Longitude station dépose": "longitude",
                "Nom station dépose": "station_name"
               }

In [3]:
def load_03_Regulation_mouvements_stations(folder_path, off_season = True):
    
    season_folder = "2023_02" if off_season else "2023_06"
    to_concat = []
    for path in glob(os.path.join(folder_path, season_folder, "*.xlsx")):
        data = pd.read_excel(path) 
        data["date"] = os.path.basename(path)[:10]
        data_drop_on = data.loc[:,["Nom station prise",  "Latitude station prise",  "Longitude station prise", 'Total', 'VAE',
       'VM', "date"]]
        data_drop_on["drop"] = "on"
        data_drop_off = data.loc[:,["Nom station dépose",  "Latitude station dépose",  "Longitude station dépose",'Total', 'VAE',
       'VM', "date"]]
        data_drop_off["drop"] = "off"
        
        to_concat.append(data_drop_on.rename(RENAME_DICT, axis=1))
        to_concat.append(data_drop_off.rename(RENAME_DICT, axis=1))
                                                      
    return pd.concat(to_concat, axis= 0)

path = os.path.join(os.path.dirname(notebook_path), "Data/03_Régulation_mouvements_stations")

dataframe_03 = load_03_Regulation_mouvements_stations(path, off_season=False)
print(dataframe_03.head())

ValueError: No objects to concatenate

In [20]:
def load_03_Regulation_mouvements_stations(folder_path, off_season = True):
    
    season_folder = "2023_02" if off_season else "2023_06"
    to_concat = []
    for path in glob(os.path.join(folder_path, season_folder, "*.xlsx")):
        data = pd.read_excel(path) 
        data["date"] = os.path.basename(path)[:10]
        data_drop_on = data.loc[:,["Nom station prise",  "Latitude station prise",  "Longitude station prise", 'Total', 'VAE',
       'VM', "date"]]
        data_drop_on["drop"] = "on"
        data_drop_off = data.loc[:,["Nom station dépose",  "Latitude station dépose",  "Longitude station dépose",'Total', 'VAE',
       'VM', "date"]]
        data_drop_off["drop"] = "off"
        
        to_concat.append(data_drop_on.rename(RENAME_DICT, axis=1))
        to_concat.append(data_drop_off.rename(RENAME_DICT, axis=1))
                                                      
    return pd.concat(to_concat, axis= 0)

path = os.path.join(os.path.dirname(notebook_path), "Data/03_Régulation_mouvements_stations")

dataframe_03 = load_03_Regulation_mouvements_stations(path, off_season=False)
print(dataframe_03.head())

                            station_name   latitude  longitude Total VAE  VM  \
0  Westermeyer - Paul Vaillant-Couturier  48.819116   2.396664    13   1  12   
1          Gare RER - Maurice Grandcoing  48.813881   2.392429     8   1   7   
2                     Charles Delescluze  48.852422   2.380632    16   3  13   
3                    Place Marcel Cachin  48.814114   2.390908     8   1   7   
4          Gare RER - Maurice Grandcoing  48.813881   2.392429     9   0   9   

         date drop  
0  2023-06-01   on  
1  2023-06-01   on  
2  2023-06-01   on  
3  2023-06-01   on  
4  2023-06-01   on  


In [5]:
def load_04_Regulation_mouvements_ateliers(folder_path, off_season = True, shop_to_station=True):
    
    season_folder = "2023_02" if off_season else "2023_06"
    shop_folders = ["ateliers-vers-stations" , "stations-vers-ateliers"]
    
    to_concat = []
    for shop_folder in shop_folders:
        drop_on = "on" if shop_folder == "ateliers-vers-stations" else "off"
        print(os.path.join(folder_path, shop_folder, season_folder, "*.xlsx"))
        for path in glob(os.path.join(folder_path, shop_folder, season_folder, "*.xlsx")):
            data = pd.read_excel(path) 
            data["date"] = os.path.basename(path)[:10]
            data["drop"] = drop_on
            to_concat.append(data.rename(RENAME_DICT, axis=1))

    return  pd.concat(to_concat, axis= 0)

path = os.path.join(os.path.dirname(notebook_path), "Data/04_Régulation_mouvements_ateliers")

dataframe_04 = load_04_Regulation_mouvements_ateliers(path, off_season=False, shop_to_station=True)
print(dataframe_04.head())

C:\Users\ewend\Desktop\Dev\Projets git\Sujet-2-Etudes-de-cas\graphes\../Data/04_Régulation_mouvements_ateliers\ateliers-vers-stations\2023_06\*.xlsx
C:\Users\ewend\Desktop\Dev\Projets git\Sujet-2-Etudes-de-cas\graphes\../Data/04_Régulation_mouvements_ateliers\stations-vers-ateliers\2023_06\*.xlsx


ValueError: No objects to concatenate

## Functions to load and display maps

In [9]:
def create_map(dataframe, zoom_start=15):
    median = dataframe[['latitude', 'longitude']].median()
    m = folium.Map(location=(median['latitude'], median['longitude']),
                   zoom_start=zoom_start,
                   zoom_control=False,
                   scrollWheelZoom=False)
    return m

def add_marker(map, latitude=0.0, longitude = 0.0, nb_velibs = -1, name="", time = "2023-02-01"):

    folium.CircleMarker(
        location=[latitude, longitude],
        tooltip=name,
        popup="Not implemented yet",
        radius=2 if nb_velibs==-1 else nb_velibs/2,
        time = time
    ).add_to(m)
    
""" Markers for drop on and off dataset"""
def create_markers(dataframe):
    station_size_to_feature = defaultdict(lambda:  {
                            "type": "Feature",
                            "geometry": {
                                "type": "MultiPoint",
                                "coordinates": [],
                            },
                            "properties": {
                                "times": [],    
                                "icon": "circle",
                                "iconstyle": {"radius": 13, "color": "green"}
                            },
                        })
    for (date, station_name), g in dataframe.groupby(["date", "station_name"]):
        lat = g['latitude'].iloc[0]
        lon = g['longitude'].iloc[0]
        nb_velibs = g['Total'].iloc[0]
        drop = g['drop'].iloc[0] 
        station_size_to_feature[nb_velibs]["geometry"]["coordinates"].append([lon, lat])
        station_size_to_feature[nb_velibs]["properties"]["times"].append(date)
        station_size_to_feature[nb_velibs]["properties"]["iconstyle"]["radius"] = nb_velibs/2
        station_size_to_feature[nb_velibs]["properties"]["iconstyle"]["color"] = "red" if drop == "on" else "green"
    return station_size_to_feature

dataframe = pd.read_csv("../Data/01_Courses_usagers/2023_02/All_data_courses.csv", sep=";").loc[:, ['Latitude_start', 'longitude_start']]
dataframe = dataframe.rename(columns={"Latitude_start": "latitude", "longitude_start": "longitude"})
create_markers(dataframe.iloc[:2])

ValueError: too many values to unpack (expected 2)

## Display data

In [25]:
def map_stations_drop(data_stations):
    m = create_map(data_stations, zoom_start = 12)
    features = list(create_markers(data_stations).values())

    data = {
            "type": "FeatureCollection",
            "features": features,
        }

    tgj = TimestampedGeoJson(data, add_last_point=False,
                                   period='P1D', 
                                   duration='P1D')
    m.add_child(tgj)

    return m


## Load dataframes

In [34]:
path_03 = os.path.join(os.path.dirname(notebook_path), "Data/03_Régulation_mouvements_stations")
path_04 = os.path.join(os.path.dirname(notebook_path), "Data/04_Régulation_mouvements_ateliers")

dataframe_03 = load_03_Regulation_mouvements_stations(path_03, off_season=True)
dataframe_04 = load_04_Regulation_mouvements_ateliers(path_04, off_season=True)

/drive/notebooks/Data/04_Régulation_mouvements_ateliers/ateliers-vers-stations/2023_02/*.xlsx
/drive/notebooks/Data/04_Régulation_mouvements_ateliers/stations-vers-ateliers/2023_02/*.xlsx


In [35]:
map_stations_drop(dataframe_03)

In [36]:
map_stations_drop(dataframe_04)

After interacting with the system, we can take the result and perform further computations. In this case, we compute the average positions in \\(x\\), \\(y\\) and \\(z\\).

In [ ]:
xyz_avg = x_t.mean(axis=1)

In [ ]:
xyz_avg.shape

Creating histograms of the average positions (across different trajectories) show that, on average, the trajectories swirl about the attractors.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
plt.hist(xyz_avg[:,0])
plt.title('Average $x(t)$');

In [ ]:
plt.hist(xyz_avg[:,1])
plt.title('Average $y(t)$');